## Buoyancy Lab

We'll start by redefining our error calculating toolkit.

In [41]:
import numpy as np
from typing import Iterable, Tuple, Union

# Define our rules...

def rule_1(c, error_val):
    return abs(c) * error_val

def rule_2(c, val, error_val, power):
    return abs(c * power * val ** (power - 1)) * error_val

def rule_3(*err_vals: Iterable[float]) -> float:
    """
    Calculate rule 4 from the paper 'Treatment of Data'.
    
    @param err_vals: A list of parameters, being the errors in each value. It is assumed they were summed 
                     together to get the final value.
    
    @returns: The error of all of the sum of the values...
    """
    total = 0
    
    for err in err_vals:
        total += err ** 2
    
    return np.sqrt(total)


def rule_4(value: float, *error_list: Iterable[Tuple[float, float, float]]) -> float:
    """
    Calculate rule 4 from the paper 'Treatment of Data'.
    
    @param value: The value of the thing we are trying to calculate the error of.
    @param error_list: A list of length 3 tuples. 
                       Each tuple should contain:
                       - A float: A value in the error formula.
                       - A float: The measured error in the above value.
                       - A float: The power of the above value in the multiplicative formula.

    @returns: A float, being the error in 'value'.
    """
    total = 0
    
    for x, x_err, power in error_list:
        total += (power * (x_err / x)) ** 2
        
    return abs(value) * np.sqrt(total)

# Adding this to the toolkit, checks if two values "agree" with each other.... 
FloatVec = Union[float, np.ndarray] 
BoolVec = Union[bool, np.ndarray]

def values_agree(val_1: FloatVec, err_1: FloatVec, val_2: FloatVec, err_2: FloatVec) -> BoolVec:
    """
    Determines if the values in 2 vectors(arrays) agree with each other, given there uncertainty values. 
    
    @param val_1: The 1st array of values.
    @param err_1: The uncertainty values for the 1st vector.
    @param val_2: The 2nd array of values.
    @param err_2: The uncertainty values for the 2st vector. 
    
    @returns: A vector of booleans, being whether each value agrees with the other.
    """
    r11, r12 = val_1 - err_1, val_1 + err_1
    r21, r22 = val_2 - err_2, val_2 + err_2
    
    r11 <= r21
    
    return ((r11 <= r21) & (r21 <= r12)) | ((r11 <= r22) & (r22 <= r12))


We'll also define a function for pretty printing our results.

In [42]:
def print_vals(vals, errs, units, msgs):
    for val, err, unit, msg in zip(vals, errs, units, msgs):
        print(f"{msg}: {val} ± {err} {unit}")
        
messages = [
    "Buoyancy Force for Section 1",
    "Buoyancy Force for Sections 1 & 2",
    "Buoyancy Force for Sections 1-3",
    "Buoyancy Force for Entire Cylinder"
]

units = ["N"] * 4

## Part 1 (Theoretical Buoyancy)

In [43]:
g = 9.8 # Accel due to gravity...

mass_water = (483.4 - 161.7) / 1000 # grams -> kg
mass_water_err = rule_3(0.1 / 1000, 0.1 / 1000)
print(mass_water)
print(mass_water_err)

volume_water = 326 / 1e6 # mL -> m^3
volume_water_err = 1 / 1e6 # mL -> m^3

water_d = mass_water / volume_water
water_d_err = rule_4(water_d, (mass_water, mass_water_err, 1), (volume_water, volume_water_err, -1))

heights = np.array([1.915, 3.83, 5.705, 7.72]) / 100 # cm -> m
heights_err = 0.05 / 1000 # mm -> m

r = (2.53 / 2) / 100 # cm -> m
r_err = (0.05 / 2) / 1000 # mm -> m

volume_cyl = np.pi * (r ** 2) * heights
volume_cyl_err = rule_4(volume_cyl, (r, r_err, 2), (heights, heights_err, 1))

buoyancy = volume_cyl * water_d * g
buoyancy_err = rule_4(buoyancy, (volume_cyl, volume_cyl_err, 1), (water_d, water_d_err, 1))

print_vals(buoyancy, buoyancy_err, units, messages)

0.3217
0.0001414213562373095
Buoyancy Force for Section 1: 0.09310204888474237 ± 0.0005270156809125469 N
Buoyancy Force for Sections 1 & 2: 0.18620409776948474 ± 0.0009662863339913572 N
Buoyancy Force for Sections 1-3: 0.2773614563380967 ± 0.001414098444625251 N
Buoyancy Force for Entire Cylinder: 0.37532523101316506 ± 0.0019006802208185595 N


## Part 2 (Experimental Buoyancy)

In [44]:
total_w = (111.3 / 1000) * 9.8 
total_w_err = (0.1 / 1000) * 9.8

apparent_w = (np.array([102.1, 91.9, 82.6, 72.6]) / 1000) * 9.8
apparent_w_err = (0.1 / 1000) * 9.8

buoyancy2 = total_w - apparent_w
buoyancy2_err = rule_3(total_w_err, apparent_w_err)

print_vals(buoyancy2, [buoyancy2_err] * len(buoyancy2), units, messages)

Buoyancy Force for Section 1: 0.09016000000000002 ± 0.0013859292911256335 N
Buoyancy Force for Sections 1 & 2: 0.19011999999999984 ± 0.0013859292911256335 N
Buoyancy Force for Sections 1-3: 0.28126000000000007 ± 0.0013859292911256335 N
Buoyancy Force for Entire Cylinder: 0.37926000000000004 ± 0.0013859292911256335 N


## Questions Section

__1) Write your measurements of the volume and mass of the water in your beaker including the uncertainty. (2 points)__

I got a mass for the water of $321.70 \pm 0.14\text{ }g$, and a volume for the water of $326.00 \pm 1.00\text{ }mL$.

__2) Write your measurement of the full length(height) of the cylinder including the uncertainty. (2 points)__

I got a measurement for the height of the cylinder of $7.720 \pm 0.005\text{ }cm$.

### Results

__3) What is the value of the buoyant force $F_b$ you calculated in Part I when the cylinder was half submerged including the uncertainty.  Show your work.  (6 points)__

Work for computing $F_b$ in part 1 can be seen in the code in the above sections. I got a result for $F_b$ when the cylinder was half submerged in part 1 of $0.186 \pm 0.001\text{ }N$. 

__4) What is the value of the buoyant force $F_b$ you found in Part II when the cylinder was half submerged including the uncertainty.  Show your work.  (4 points)__

Work for computing $F_b$ in part 2 can be seen in the code in the above sections. I got a result for $F_b$ when the cylinder was half submerged in part 2 of $0.190 \pm 0.001\text{ }N$.

__5) Do your results for the forces from Parts I and II agree for when the cylinder was half submerged?  Justify your answer.  (4 points)__

The values I got for both part 1 and 2 do not agree with each other, because their error ranges to not overlap. In this case, $F_b$ for part 1 ranges from $0.185 \text{ }N$ to $0.187 \text{ }N$, while the value computed for $F_b$ in part 2 ranges from $0.189 \text{ }N$ to $0.192 \text{ }N$.

__6) Do you results for Parts I and II agree for when the cylinder was $\frac{1}{4}$ submerged, $\frac{3}{4}$ submerged and fully submerged? (4 points)__

None of my results agree with each other for parts I and II, including when the cylinder was $\frac{1}{4}$ submerged, $\frac{3}{4}$ submerged, and fully submerged. This can be seen in the code below, which checks to see if any of the error ranges overlap.

In [45]:
# Extreme sadness :(... (None of my values agree)
print()
print("Checking if values agree...")
results = values_agree(buoyancy, buoyancy_err, buoyancy2, buoyancy2_err)
for msg, result in zip(messages, results):
    print(f'\t{msg}: Parts I and II{"" if(result) else " DO NOT"} agree with each other.')
print(f"Percent of Agreeing Values: {np.mean(results) * 100:.02f}%")


Checking if values agree...
	Buoyancy Force for Section 1: Parts I and II DO NOT agree with each other.
	Buoyancy Force for Sections 1 & 2: Parts I and II DO NOT agree with each other.
	Buoyancy Force for Sections 1-3: Parts I and II DO NOT agree with each other.
	Buoyancy Force for Entire Cylinder: Parts I and II DO NOT agree with each other.
Percent of Agreeing Values: 0.00%


### Concepts

__7) If we had used ethyl alcohol, with a density of approximately $800 \frac{kg}{m^3}$, instead of water for this experiment, would the buoyant forces have been greater, smaller, or the same as the ones that you found?  Justify your answer.  (3 points)__

If ethyl alcohol had been used instead of water, the buoyant forces would have been smaller then the ones computed in this experiment. We can see why by looking at the formula for the buoyant force:

$$
F_b = \text{weight of displaced fluid} = mg = \rho Vg
$$

In this formula, if the volume and acceleration due to gravity are kept constant, the buoyant force and the density of the fluid become directly proportional. This means raising the fluid density will increase the buoyant force, while decreasing the fluid density will decrease the buoyant force. Since the density of ethyl alcohol is less than water, the buoyant force would be less.